# 3.3 线性回归的简洁实现

In [10]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.4.0


## 3.3.1 生成数据集

In [2]:
num_inputs = 2        # 2个特征
num_examples = 1000   # 1000个数据
true_w = [2, -3.4]    # w，b的真实值
true_b = 4.2
# 生成1000x2的正态随机数
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
# y=wx+b+e 加上随机噪声生成数据的标签
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b 
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [3]:
# PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [4]:
# 打印出一个batch
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.4169,  0.2818],
        [ 0.2792, -0.5584],
        [ 0.3924, -0.6813],
        [-1.3038, -0.0930],
        [ 0.1400,  0.8914],
        [ 1.4269,  2.8872],
        [-1.5373, -0.1403],
        [-0.0302,  2.9876],
        [ 0.8121,  0.7475],
        [-0.7700,  1.6500]]) 
 tensor([ 4.0771,  6.6406,  7.2967,  1.9111,  1.4535, -2.7647,  1.6083, -6.0152,
         3.2933, -2.9417])


## 3.3.3 定义模型

nn就是利用autograd来定义模型的。nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。

In [5]:
# 使用nn定义一个线性回归模型
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()     # 继承nn.Module类，super解决nn中的初始化参数继承问题
        self.linear = nn.Linear(n_feature, 1) # nn.Linear(1000,1)构造线性层

    def forward(self, x):                     # 构造前向传播
        y = self.linear(x)                  
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [11]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1) # 输入特征2 输出标签1
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [13]:
# 通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1455,  0.3597]], requires_grad=True)
Parameter containing:
tensor([0.0983], requires_grad=True)


注意：**torch.nn**仅支持输入一个**batch**的样本不支持单个样本输入，如果只有单个样本，可使用**input.unsqueeze(0)**来添加一维。

## 3.3.4 初始化模型参数

In [8]:
from torch.nn import init  # 利用init模块

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [15]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1455,  0.3597]], requires_grad=True)
Parameter containing:
tensor([0.0983], requires_grad=True)


注：如果这里的net是用3.3.3节一开始的代码自定义的，那么上面代码会报错，net[0].weight应改为net.linear.weight，bias亦然。因为net[0]这样根据下标访问子模块的写法只有当net是个ModuleList或者Sequential实例时才可以，详见4.1节。

## 3.3.5 定义损失函数

In [16]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [17]:
# torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率，这在finetune时经常用到
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

In [13]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [18]:
# 通过调用optim实例的step函数来迭代模型参数。
# 按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均

num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000351
epoch 2, loss: 0.000034
epoch 3, loss: 0.000074


In [19]:
# 我们从net获得需要的层，并访问其权重（weight）和偏差（bias）
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 1.9989, -3.3995]])
4.2 tensor([4.2001])
